In [1]:
import os
import numpy as np
from lib.utils import model_results_from_npz

CIFAR100_CLASSES = [
    'apple', 'aquarium_fish', 'baby', 'bear', 'beaver', 
    'bed', 'bee', 'beetle', 'bicycle', 'bottle', 'bowl', 
    'boy', 'bridge', 'bus', 'butterfly', 'camel', 'can', 
    'castle', 'caterpillar', 'cattle', 'chair', 'chimpanzee', 
    'clock', 'cloud', 'cockroach', 'couch', 'crab', 'crocodile', 
    'cup', 'dinosaur', 'dolphin', 'elephant', 'flatfish', 'forest', 
    'fox', 'girl', 'hamster', 'house', 'kangaroo', 'keyboard', 'lamp', 
    'lawn_mower', 'leopard', 'lion', 'lizard', 'lobster', 'man', 
    'maple_tree', 'motorcycle', 'mountain', 'mouse', 'mushroom', 
    'oak_tree', 'orange', 'orchid', 'otter', 'palm_tree', 'pear', 
    'pickup_truck', 'pine_tree', 'plain', 'plate', 'poppy', 'porcupine', 
    'possum', 'rabbit', 'raccoon', 'ray', 'road', 'rocket', 'rose', 'sea', 
    'seal', 'shark', 'shrew', 'skunk', 'skyscraper', 'snail', 'snake', 
    'spider', 'squirrel', 'streetcar', 'sunflower', 'sweet_pepper', 'table', 
    'tank', 'telephone', 'television', 'tiger', 'tractor', 'train', 'trout', 
    'tulip', 'turtle', 'wardrobe', 'whale', 'willow_tree', 'wolf', 'woman', 'worm'
]
NUM_CLASSES = len(CIFAR100_CLASSES)
SEEDS = [42, 602, 311, 637, 800, 543, 969, 122, 336, 93]
EPOCHS = 50
REMOVED_CLASSES = [
    4, 7, 6, 11, 14, 15, 19, 24, 23, 25, 29, 30, 
    31, 36, 35, 39, 44, 41, 49, 48, 51, 53, 59, 57, 60, 61, 
    68, 67, 72, 71, 76, 77, 80, 84, 88, 85, 92, 94, 99, 97] # CIFAR-100


DIR_RETRAIN = "../results/cifar100/"
DIR_REDIS   = "../results/redis_cifar100/"

best_match = {"seed": None, "removed_class": None, "match_ratio": -1.0}

for seed in SEEDS:
    for removed_class in REMOVED_CLASSES:
        retrain_path = os.path.join(DIR_RETRAIN, f"cifar100_resnet18_s{seed}_e{EPOCHS}_r{removed_class}.npz")
        redis_path   = os.path.join(DIR_REDIS, f"cifar_resnet_s{seed}_e{EPOCHS}_rd{removed_class}.npz")

        if not os.path.exists(retrain_path) or not os.path.exists(redis_path):
            continue

        model_retrain = model_results_from_npz(retrain_path, NUM_CLASSES)
        model_redis   = model_results_from_npz(redis_path, NUM_CLASSES)

        # Concatenate all class predictions for fair comparison
        retrain_preds = np.concatenate([model_retrain.preds[cls] for cls in range(NUM_CLASSES)])
        redis_preds   = np.concatenate([model_redis.preds[cls] for cls in range(NUM_CLASSES)])

        match_ratio = np.mean(retrain_preds == redis_preds)

        if match_ratio > best_match["match_ratio"]:
            best_match.update({"seed": seed, "removed_class": removed_class, "match_ratio": match_ratio})

print("Best matching seed and removed class:")
print(best_match)


Best matching seed and removed class:
{'seed': 969, 'removed_class': 35, 'match_ratio': np.float64(0.7855)}
